## Hello world

Here we want to demonstrate how to use solve Linear Programming problems using Python.

In [1]:
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, GLPK

In [2]:
# Example from here: https://realpython.com/linear-programming-python/#example-1
obj = [-1, -2]
#      ─┬  ─┬
#       │   └┤ Coefficient for y
#       └────┤ Coefficient for x

lhs_ineq = [
    [ 2, 1],  # Red constraint left side
    [-4, 5],  # Blue constraint left side
    [1, -2],  # Yellow constraint left side
]

rhs_ineq = [
    20,  # Red constraint right side
    10,  # Blue constraint right side
    2,  # Yellow constraint right side
]

lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]  # Green constraint right side

bnd = [
    (0, float("inf")),  # Bounds of x
    (0, float("inf")),  # Bounds of y
]

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="revised simplex")
opt

     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

In [3]:
# Same example, but with PuLP
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# How to define objective function
expression = 2 * x + 4 * y
print(type(expression))

# How to define constraints
constraint = 2 * x + 4 * y >= 8
print(type(constraint))

# Define the model, with objective functions and constraints
# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")
# Add the objective function to the model
obj_func = x + 2 * y
model += obj_func
# Add the same objective function to the model, with different method
model += lpSum([x, 2 * y])

model

<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpConstraint'>


/Users/changyaochen/.local/share/virtualenvs/Coursera-operation-research-1-rOQX_doW/lib/python3.8/site-packages/pulp/pulp.py:1353: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


small-problem:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

In [4]:
# Solve the problem with the default solver
model.solve()
model.objective.value()

16.8181817

In [5]:
# Solve the problem with a different solver
model.solve(solver=GLPK(msg=False))

1

## 0-1 Knapsack example

Here we want to show how to use `pulp` solve the famous 0-1 Knapsack problem. The example comes from the week 1 notes of the course.
![title](./knapsack_example.png)

In [6]:
MAX_WEIGHT = 5.0
knapsack_model = LpProblem(name="knapsack", sense=LpMaximize)

# Decision variables
x = {i: LpVariable(name=f"x{i}", cat="Binary") for i in range(7)}

# Objective function
values = [6, 5, 4, 4, 3, 4, 1]
obj_func = lpSum([v * x[i] for i, v in enumerate(values)])
knapsack_model += obj_func

# Constraints
weights = [0.5, 1.5, 0.4, 1, 1.1, 1.6, 0.8]
constraint = lpSum([w * x[i] for i, w in enumerate(weights)]) <= MAX_WEIGHT
knapsack_model += (constraint, "Total weight constraint")

knapsack_model

knapsack:
MAXIMIZE
6*x0 + 5*x1 + 4*x2 + 4*x3 + 3*x4 + 4*x5 + 1*x6 + 0
SUBJECT TO
Total_weight_constraint: 0.5 x0 + 1.5 x1 + 0.4 x2 + x3 + 1.1 x4 + 1.6 x5
 + 0.8 x6 <= 5

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer
0 <= x6 <= 1 Integer

In [7]:
status = knapsack_model.solve()
print(f"The status of the solver is {status}")
print(f"The optimal value is {knapsack_model.objective.value()}")

print("The optimal decision is:")
for var in knapsack_model.variables():
    print(f"{var.name}: {var.value()}")

The status of the solver is 1
The optimal value is 23.0
The optimal decision is:
x0: 1.0
x1: 1.0
x2: 1.0
x3: 1.0
x4: 0.0
x5: 1.0
x6: 0.0


In [8]:
# DP solution